In [28]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
 

In [29]:
#사용할 파일 임포트
df = pd.read_csv("../data/us_new_born_2million.csv")

In [30]:
df.dtypes

weight_pounds         float64
state                  object
year                    int64
month                   int64
child_race            float64
plurality               int64
mother_race           float64
mother_age              int64
cigarette_use          object
cigarettes_per_day    float64
alcohol_use            object
drinks_per_week       float64
father_race           float64
father_age              int64
is_male                  bool
gestation_weeks         int64
dtype: object

In [31]:
# 각 값이 빈 값 여부 판단
# 비어있다면 True ,  안 비어있다면 False
df.isna()
# sum 과 함께 사용하면, 빈값, 즉 True의 개수를 return 해준다.
df.isna().sum()
# shape를 이용하면 , (row수, col수)로 return 해준다.
df.shape
# 아래와 같이 작성하면 True의 비율을 알 수 있다.
df.isna().sum()/df.shape[0]
# 아래와 같이 작성하면, True 의 비율이 0.7 이상인 것을 구할 수 있다. (이상이면 True, 미만이면 False)
df.isna().sum()/df.shape[0] > 0.7



weight_pounds         False
state                 False
year                  False
month                 False
child_race             True
plurality             False
mother_race           False
mother_age            False
cigarette_use         False
cigarettes_per_day     True
alcohol_use           False
drinks_per_week        True
father_race           False
father_age            False
is_male               False
gestation_weeks       False
dtype: bool

In [32]:
df.isna().sum()[df.isna().sum()/df.shape[0] > 
                ((df.isna().sum()/df.shape[0]).mean()+ 
                (df.isna().sum()/df.shape[0]).std())].index

    

Index(['child_race', 'cigarettes_per_day', 'drinks_per_week'], dtype='object')

In [33]:
#weigt , state 칼럼 삭제하기.
# df에 삭제시킨 것을 바로 적용시키고 싶다면, 'inplace = True' 옵션을 추가해주면 된다.
df.drop(['weight_pounds','state'],axis =1 )

,year,month,child_race,plurality,mother_race,mother_age,cigarette_use,cigarettes_per_day,alcohol_use,drinks_per_week,father_race,father_age,is_male,gestation_weeks
0,2003,9,NaN,1,1.0,34,False,NaN,False,NaN,1.0,39,False,39
1,2003,1,NaN,1,1.0,37,True,15.0,True,0.0,1.0,31,True,41
2,2003,12,NaN,1,1.0,30,False,NaN,False,NaN,1.0,38,False,40
3,2003,4,NaN,1,7.0,39,False,NaN,False,NaN,1.0,39,True,40
4,2004,2,NaN,1,1.0,38,False,NaN,False,NaN,1.0,39,True,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998241,2003,9,NaN,1,38.0,35,False,NaN,False,NaN,99.0,99,True,40
1998242,2003,1,NaN,1,2.0,32,True,6.0,True,0.0,99.0,99,True,38
1998243,2004,8,NaN,1,2.0,30,False,NaN,False,NaN,99.0,48,False,39
1998244,2004,12,NaN,2,2.0,29,False,NaN,False,NaN,99.0,99,False,40


In [34]:
# 추출한 index와 drop을 한번에 작성하기 !
# 빈데이터가 na값이 

df.drop(df.isna().sum()[df.isna().sum()/df.shape[0] > 
                ((df.isna().sum()/df.shape[0]).mean()+ 
                (df.isna().sum()/df.shape[0]).std())].index , axis =1 , inplace= True)

In [35]:
df.columns.size

13

In [36]:
df.head()

,weight_pounds,state,year,month,plurality,mother_race,mother_age,cigarette_use,alcohol_use,father_race,father_age,is_male,gestation_weeks
0,7.813183,GA,2003,9,1,1.0,34,False,False,1.0,39,False,39
1,8.000575,GA,2003,1,1,1.0,37,True,True,1.0,31,True,41
2,7.251004,GA,2003,12,1,1.0,30,False,False,1.0,38,False,40
3,9.568062,GA,2003,4,1,7.0,39,False,False,1.0,39,True,40
4,7.187070,GA,2004,2,1,1.0,38,False,False,1.0,39,True,40


In [37]:
# 행의 nan 갯수가 30% 이상인 행 추출
df.isna().sum(axis=1)[df.isna().sum(axis=1) /df.columns.size >0.3].index


Int64Index([   6990,    6991,    6992,    6993,    6994,    6995,    6996,
               6997,    6998,    6999,
            ...
            1998156, 1998157, 1998158, 1998159, 1998160, 1998161, 1998162,
            1998163, 1998164, 1998165],
           dtype='int64', length=226858)

In [38]:
# 추출한 행 삭제하기
df.drop(df.isna().sum(axis=1)[df.isna().sum(axis=1) /df.columns.size >0.3].index,axis=0 , inplace=True)

In [39]:
df.shape

(1771388, 13)

In [40]:
# .copy()로 복사해야, df1과 df가 독립적으로 수행이된다.(서로 다른 변수가 된다.)
df1 = df.copy()

In [84]:
df1.weight_pounds.fillna(df1.weight_pounds.mean(),inplace = True)


In [85]:
df.weight_pounds[df1.is_male & df.weight_pounds.isna()] = df.loc[df1.is_male,'weight_pounds']

<ipython-input-85-6b227b849999>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.weight_pounds[df1.is_male & df.weight_pounds.isna()] = df.loc[df1.is_male,'weight_pounds']


In [86]:
df1 = np.where(~df.weight_pounds.isna() , df.weight_pounds,
        np.where(df.is_male  , 
            df.loc[df.is_male,'weight_pounds'] ,  df.loc[~df.is_male,'weight_pounds']))

ValueError: operands could not be broadcast together with shapes (1771388,) (907224,) (864164,) 

In [44]:
df1.loc[df1.cigarette_use.isna(),:]
# cigarette_use | alcohol_use ! = nan 인 행 추출
data = df1.loc[~df1.cigarette_use.isna() | ~df1.alcohol_use.isna()]
cols = ['plurality','mother_race','mother_age','father_race','father_age','cigarette_use']
data = data[cols]
# cigarette_use 를 int32로 형변환
data.cigarette_use = data.cigarette_use.astype('int32')
# 분석을 위해 x,y로 구분 
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
# train, test로 나누어줌
# X_train, X_test ,Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
# rf = RandomForestClassifier()
# rf.fit(X_train,Y_train)

# 아래 3줄을 주석을 풀어야함... 그런데 너무 오래걸림.. ( Tensor꼭 성공하자...)



In [45]:
# 신뢰도 점수!
rf.score(X_test,Y_test)

NameError: name 'rf' is not defined

In [46]:
#데이터 확인
df1.alcohol_use.value_counts()
#분석을 통해 na를 imputation해준다.
data = df1.loc[~df1.alcohol_use.isna(),:]
data

,weight_pounds,state,year,month,plurality,mother_race,mother_age,cigarette_use,alcohol_use,father_race,father_age,is_male,gestation_weeks
0,7.813183,GA,2003,9,1,1.0,34,False,False,1.0,39,False,39
1,8.000575,GA,2003,1,1,1.0,37,True,True,1.0,31,True,41
2,7.251004,GA,2003,12,1,1.0,30,False,False,1.0,38,False,40
3,9.568062,GA,2003,4,1,7.0,39,False,False,1.0,39,True,40
4,7.187070,GA,2004,2,1,1.0,38,False,False,1.0,39,True,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998241,7.228958,HI,2003,9,1,38.0,35,False,False,99.0,99,True,40
1998242,6.375769,IL,2003,1,1,2.0,32,True,True,99.0,99,True,38
1998243,6.375769,MD,2004,8,1,2.0,30,False,False,99.0,48,False,39
1998244,6.750554,MS,2004,12,2,2.0,29,False,False,99.0,99,False,40


In [47]:
# 필요한 데이터 추출(정리)
cols = ['plurality','mother_race','mother_age','cigarette_use','father_race','father_age','alcohol_use']
data = data[cols]
data.isna()


,plurality,mother_race,mother_age,cigarette_use,father_race,father_age,alcohol_use
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
1998241,False,False,False,False,False,False,False
1998242,False,False,False,False,False,False,False
1998243,False,False,False,False,False,False,False
1998244,False,False,False,False,False,False,False


In [48]:
data.isna().sum()
data.dtypes
#  cigarette_use, alcohol_use 이 Object이기 때문에 형변환
data.cigarette_use = data.cigarette_use.astype('int32')
data.alcohol_use = data.alcohol_use.astype('int32')
data.dtypes

C:\Users\painc\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


plurality          int64
mother_race      float64
mother_age         int64
cigarette_use      int32
father_race      float64
father_age         int64
alcohol_use        int32
dtype: object

In [49]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
ind = np.random.choice(X.index,X.index.size,replace=False)
print(ind.size)
print(X.shape)

1492553
(1492553, 6)


In [50]:
tr_ind = ind[ : np.ceil(ind.size*0.8).astype('int32')]
ta_ind = ind[np.ceil(ind.size*0.8).astype('int32'):]

In [51]:
X_train = X.loc[tr_ind,:]
X_test =  X.loc[ta_ind,:]
Y_train = Y[tr_ind]
Y_test =  Y[ta_ind]


In [53]:
# from sklearn.svm import SVC
# sv_model = SVC(C =10 , kernel = 'rbf')
# sv_model.fit(X_train,Y_train)
# sv_model.score(X_test,Y_test)
# svm은 너무 오래걸린다..
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,Y_train)
lr.score(X_test,Y_test)


C:\Users\painc\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1.0

In [61]:
x = df1.loc[df1.alcohol_use.isna(),cols]

In [63]:
pred = lr.predict(x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [72]:
df1.isna().sum()

weight_pounds         887
state                   0
year                    0
month                   0
plurality               0
mother_race             0
mother_age              0
cigarette_use      278835
alcohol_use        278835
father_race             0
father_age              0
is_male                 0
gestation_weeks         0
dtype: int64

In [78]:
cols = ["year","month","plurality","mother_race","mother_age","cigarette_use","alcohol_use","father_race","father_age","gestation_weeks","is_male"]
data = df1[cols]
data.alcohol_use =data.alcohol_use.astype('int32')
data.cigarette_use =data.cigarette_use.astype('int32')
data.is_male =data.is_male.astype('int32')
data

ValueError: cannot convert float NaN to integer